In [3]:
from glob import glob
import os
import json

In [4]:
# TODO ADD CLASS DETAILS

root_path = "../data/a9_temporal"

for split in ["train", "val", "test"]:
    if not os.path.exists(os.path.join(root_path, split)):
        continue

    img_label_s1_folder = os.path.join(
        root_path, split, "labels_images", "s110_camera_basler_south1_8mm"
    )
    img_label_s1_paths = sorted(glob(os.path.join(img_label_s1_folder, "*")))

    total_not_only_prev = 0
    total_not_only_next = 0
    total_not_both = 0

    data = {}

    for x in img_label_s1_paths:
        img_label_s1_json_path = os.path.join(x)
        json_data = None
        with open(img_label_s1_json_path, "r") as f:
            json_data = json.load(f)
            frame_idx = list(json_data["openlabel"]["frames"].keys())[0]
            frame_properties = json_data["openlabel"]["frames"][frame_idx]["frame_properties"]
            token = frame_properties["token"]
            prev = frame_properties["prev"]
            next = frame_properties["next"]
            scene_token = frame_properties["scene_token"]
            frame_idx = frame_properties["frame_idx"]
            if prev is None and next is None:
                total_not_both += 1
            elif prev is None:
                total_not_only_prev += 1
            elif next is None:
                total_not_only_next += 1

            data[token] = (prev, next, scene_token, frame_idx)

    chain_results = {}

    for token, (_, _, scene_token, _) in data.items():
        if scene_token not in chain_results:
            chain_results[scene_token] = {
                "total": 0,
                "frames": [token]
            }
        else:
            chain_results[scene_token]["frames"].append(token)

    for token, (prev, next, scene_token, frame_idx) in data.items(): 
        chain_results[scene_token]["total"] = len(chain_results[scene_token]["frames"])

    print("\n", "-" * 15, "split: ", split, "-" * 15)
    chain_results = sorted(chain_results.items(), key=lambda x:int(x[1]["total"]))

    print("No. Sequences with following number of frames: ")
    for x, y in sorted(chain_results):
        print("\tscene token", x, "\ttotal no frames: \t", y["total"])

    print("\nNumber of sequences: \t\t", len(chain_results))
    print("Samples with no prev: \t\t", total_not_only_prev)
    print("Samples with no next: \t\t", total_not_only_next)
    print("Samples with no prev nor next: \t", total_not_both)


 --------------- split:  train ---------------
No. Sequences with following number of frames: 
	scene token 05043a60ef3d4d0f83a19a4618933555 	total no frames: 	 90
	scene token 0c35754c0898412b989a29dd0ba13424 	total no frames: 	 120
	scene token 18ae50f824cd4ec9b108f296b3ae2070 	total no frames: 	 60
	scene token 341cf65c155c47d58178bbfd4d7b3fb8 	total no frames: 	 60
	scene token 357503b6832e435d9453761ec08a91a0 	total no frames: 	 30
	scene token 68f29212061e4c0f820c90711aee8b5d 	total no frames: 	 90
	scene token 6bfc7d7aa62042619c750b5875e2db8f 	total no frames: 	 60
	scene token 7f1dd6eac5204075995899d71ee9af74 	total no frames: 	 120
	scene token 94f24f15c9d2428cb6510229579cad29 	total no frames: 	 210
	scene token a4c98c1cda4744469018bb8280b488fe 	total no frames: 	 180
	scene token a5ade10d818e468881bb87aac2a445fd 	total no frames: 	 180
	scene token ac4f7646a4964e2081afac53d0c2c51a 	total no frames: 	 90
	scene token d12f2f7292844eec9d70ca44b1a44828 	total no frames: 	 30
	s